In [0]:
df_hr = spark.read.format('csv').option("header", "true").option("inferSchema", "True").load("/Volumes/workspace/default/srirama/Sales_SalesOrderDetail.csv")

In [0]:
# Import all PySpark SQL functions and data types
from pyspark.sql.functions import *
from pyspark.sql.types import *

# Read CSV file as DataFrame with header and auto-infer schema, then select specific columns
df_sales = spark.read.format('csv').option("header", "true").option("inferSchema", "True").load("/Volumes/workspace/default/srirama/Sales_SalesOrderDetail.csv").select("SalesOrderID", "SalesOrderDetailID", "CarrierTrackingNumber", "OrderQty", "ProductID", "SpecialOfferID", "UnitPrice", "UnitPriceDiscount", "LineTotal", "ModifiedDate")

# Select columns using SQL expressions - renames SalesOrderID to so_id (others unchanged)
df_sales = df_sales.selectExpr("SalesOrderID as so_id", "SalesOrderDetailID", "CarrierTrackingNumber", "OrderQty", "ProductID", "SpecialOfferID", "UnitPrice", "UnitPriceDiscount", "LineTotal", "ModifiedDate")

# Add ModifiedYear column (extract year from ModifiedDate) + constant "sales-info" category column
df_sales = df_sales.withColumn("ModifiedYear", year("ModifiedDate")).withColumn("category", lit("sales-info"))

# Add multiple date columns at once - ModifiedDay and ModifiedMonth from ModifiedDate
df_sales = df_sales.withColumns({"ModifiedDay":day("ModifiedDate"), "ModifiedMonth":month("ModifiedDate")})

# Display formatted table view of DataFrame (Databricks-specific)
df_sales.display()

# Rename multiple columns in chain - ModifiedYear->OrderYear, ModifiedMonth->OrderMonth, ModifiedDay->OrderDay
df_sales = df_sales.withColumnRenamed("ModifiedYear", "OrderYear").withColumnRenamed("ModifiedMonth", "OrderMonth").withColumnRenamed("ModifiedDay", "OrderDay")

# Drop unwanted columns - category and OrderDay
df_sales = df_sales.drop("category","OrderDay")

# Limit to first 10 rows (creates new df_sales1)
df_sales1 = df_sales.limit(10)

# Sample 10% of data randomly (with replacement)
df_sales = df_sales.sample(fraction=0.1)

# Remove exact duplicate rows based on all columns
df_sales = df_sales.distinct()

# Remove duplicates (same as distinct() - both based on all columns)
df_sales = df_sales.dropDuplicates()

# UNION: Combines df_sales + df_sales1 (vertical append, requires same schema/columns)
df_sales_sets = df_sales.union(df_sales1)

# UNION BY NAME: Combines by column names (allows different column order)
df_sales_sets = df_sales.unionByName(df_sales1)

# INTERSECT: Returns only common rows between both DataFrames
df_sales_sets = df_sales.intersect(df_sales1)

# INTERSECT ALL: Returns common rows with multiplicity (counts duplicates)
df_sales_sets = df_sales.intersectAll(df_sales1)

# EXCEPT ALL: Returns rows in df_sales NOT in df_sales1 (preserves duplicates)
df_sales_sets = df_sales.exceptAll(df_sales1)

# Display final result
df_sales_sets.display()


In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
df_sales = spark.read.format('csv').option("header", "true").option("inferSchema", "True").load("/Volumes/workspace/default/srirama/Sales_SalesOrderDetail.csv").select("SalesOrderID", "SalesOrderDetailID", "CarrierTrackingNumber", "OrderQty", "ProductID", "SpecialOfferID", "UnitPrice", "UnitPriceDiscount", "LineTotal", "ModifiedDate")
df_sales = df_sales.selectExpr("SalesOrderID as so_id", "SalesOrderDetailID", "CarrierTrackingNumber", "OrderQty", "ProductID", "SpecialOfferID", "UnitPrice", "UnitPriceDiscount", "LineTotal", "ModifiedDate")
df_sales = df_sales.withColumn("ModifiedYear", year("ModifiedDate")).withColumn("category", lit("sales-info"))
df_sales = df_sales.withColumns({"ModifiedDay":day("ModifiedDate"), "ModifiedMonth":month("ModifiedDate")})
df_sales.display()
# df_sales = df_sales.selectExpr("ModifiedYear as OrderYear", "*")
df_sales = df_sales.withColumnRenamed("ModifiedYear", "OrderYear").withColumnRenamed("ModifiedMonth", "OrderMonth").withColumnRenamed("ModifiedDay", "OrderDay")
df_sales = df_sales.drop("category","OrderDay")
# rdd1 = df_sales.rdd
# df_sales1 = rdd1.toDF()
# df_sales = df_sales.filter(col("OrderQty") > 5)
# df_sales = df_sales.where("OrderQty > 5")
df_sales1 = df_sales.limit(10)
df_sales = df_sales.sample(fraction=0.1)
df_sales = df_sales.distinct()
df_sales = df_sales.dropDuplicates()
df_sales_sets = df_sales.union(df_sales1)
df_sales_sets = df_sales.unionByName(df_sales1)
df_sales_sets = df_sales.intersect(df_sales1)
df_sales_sets = df_sales.intersectAll(df_sales1)
# df_sales_sets = df_sales.except(df_sales1)
df_sales_sets = df_sales.exceptAll(df_sales1)
df_sales_sets.display()
# df_sales.display()
# df_sales.printSchema()